In [21]:
from xgboost import XGBClassifier

import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score

In [2]:
raw_train = pd.read_csv('train.csv', index_col='Id')
raw_test = pd.read_csv('test.csv', index_col='Id')

In [7]:
X_train = raw_train.copy().drop(['Cover_Type'], axis=1)
y_train = raw_train.copy()['Cover_Type']
X_test = raw_test.copy()

In [30]:
model = XGBClassifier(n_estimators=2500, learning_rate=0.05)

In [31]:
cv_folds = 5
scores = cross_val_score(model, X_train, y_train, cv=cv_folds, scoring='accuracy')
print('Mean accuracy: {}'.format(scores.mean()))
print('Standard error: {}'.format(scores.std() / cv_folds**0.5))

Mean accuracy: 0.7628968253968254
Standard error: 0.015547815084566265


In [32]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=2500,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [33]:
importances = pd.DataFrame({'feature':X_train.columns,
                            'importance':np.round(model.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
importances.head()

,importance
feature,
Wilderness_Area4,0.168
Elevation,0.087
Soil_Type3,0.066
Soil_Type10,0.061
Soil_Type12,0.056


In [34]:
predictions = model.predict(X_test)
preds_df = pd.DataFrame({'Id': X_test.index, 'Cover_Type': predictions})
preds_df.to_csv('submission.csv', index=False)